<a href="https://colab.research.google.com/github/EmiliaFidler/Intro_to_Comp_Ling_WS24/blob/main/final_project_comp_ling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Final Project**
---


*VU Introduction to Computational Linguistics - WS24/25*

---
**Topic**: Toxicity Detection in Social Media  
Detecting toxic or offensive language in social media, e.g. YouTube comments in Polish and English    

**Datasets**: PolishCyberbullyingDataset or hatecheck-polish for the Polish dataset and civil_comments or toxicity_classification_jigsaw for the English dataset
model: XLM-RoBERTa Base   



**Set-Up:**
Installing librarires required for the task

In [ ]:
!pip install transformers
!pip install datasets
!pip install torch
!pip install scikit-learn
!pip install matplotlib seaborn
!pip install optuna
!pip install tensorboard
!pip install pandas
!pip install numpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 28.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 364.4/364.4 kB 25.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.5/233.5 kB 22.7 MB/s eta 0:00:0

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from datasets import load_dataset

# English dataset (civil_comments)
dataset_en = load_dataset("civil_comments")

# Polish dataset (PolishCyberbullyingDataset)
dataset_pl = load_dataset("ptaszynski/PolishCyberbullyingDataset")

README.md:   0%|          | 0.00/2.13k [00:00<?, ?B/s]

v2_training_anonymized.tsv:   0%|          | 0.00/801k [00:00<?, ?B/s]

v2_test_anonymized.tsv:   0%|          | 0.00/79.5k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/10038 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [ ]:
print(dataset_en)
print(dataset_pl)

DatasetDict({
    train: Dataset({
        features: ['text', 'toxicity', 'severe_toxicity', 'obscene', 'threat', 'insult', 'identity_attack', 'sexual_explicit'],
        num_rows: 1804874
    })
    validation: Dataset({
        features: ['text', 'toxicity', 'severe_toxicity', 'obscene', 'threat', 'insult', 'identity_attack', 'sexual_explicit'],
        num_rows: 97320
    })
    test: Dataset({
        features: ['text', 'toxicity', 'severe_toxicity', 'obscene', 'threat', 'insult', 'identity_attack', 'sexual_explicit'],
        num_rows: 97320
    })
})
DatasetDict({
    train: Dataset({
        features: ['TEXT', 'GENERAL TAG', 'SPECIFIC TAG', 'CATEGORIES', 'ANY PROFANITIES?'],
        num_rows: 10038
    })
    test: Dataset({
        features: ['TEXT', 'GENERAL TAG', 'SPECIFIC TAG', 'CATEGORIES', 'ANY PROFANITIES?'],
        num_rows: 1000
    })
})


In [ ]:
train_valid_split = dataset_pl['train'].train_test_split(test_size=0.1, seed=42)

In [ ]:
train_en = dataset_en['train']
val_en = dataset_en['validation']
test_en = dataset_en['test']

train_pl = train_valid_split['train']
val_pl = train_valid_split['test']
test_pl = dataset_pl['test']

In [ ]:
print("Train size:", len(train_pl))
print("Validation size:", len(val_pl))
print("Test size:", len(test_pl))

print("Train size:", len(train_en))
print("Validation size:", len(val_en))
print("Test size:", len(test_en))

Train size: 9034
Validation size: 1004
Test size: 1000
Train size: 1804874
Validation size: 97320
Test size: 97320


Preprocessing!

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-base")
